## Data Exploration and Pre-processing:

### Import libraries and datasets: ABCD

In [1]:
import pandas as pd
from sklearn import preprocessing 
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split,GridSearchCV
import joblib
from sklearn.metrics import f1_score
%matplotlib inline

### Feature engineering:

Features: total transaction volume, number of transactions, average transaction size, transaction frequency, transaction amount variability, sales growth rate, decline in transaction volume, weekend transaction percentage, peak sales hours, transaction type distribution, average transaction growth, sales variability, sales seasonality, number of unique customers, customer retention rate, Herfindahl-Hirschman Index (HHI) of customer concentration, decline in transaction frequency, refund percentage, days since last transaction, percent of high-ticket transactions, transaction patterns on holidays or off-business hours, merchant performance relative to peers, merchant market share in district, post-loan sales growth, transaction volume volatility post-loan, change in average transaction size post-loan, sales volatility post-loan, percentage of refunded transactions post-loan, business type, business location, length of relationship, merchant size

### Examining columns and data types:

### Checking and handling missing values:

### Checking for outliers and removing them:

Balancing the dataset: